# **Movie Predictions Part 2 B**

Joe Lardie

March 2023

## **Import Data**

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [3]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2001.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [4]:
['title_akas.csv.gz','title_basics.csv.gz','title_ratings.csv.gz']

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

## **Define Function**

In [5]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [6]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', low_memory=False)

In [7]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [8]:
ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', low_memory=False)

In [9]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
2,tt0000003,6.5,1791
3,tt0000004,5.6,179
4,tt0000005,6.2,2594


In [10]:
akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', low_memory=False)

In [11]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [12]:
# Concatenate the dataframes vertically and ignore the original indices
df_concat = pd.concat([basics, ratings, akas], ignore_index=True)

In [13]:
df_concat.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Create Required Lists for loop**

In [14]:
YEARS_TO_GET = [2000,2001]

## **Define Errors list**

In [15]:
errors =[]

### **Start Outter Loop**

## **Set up Progress Bar**

In [16]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='startYear', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
             json.dump([{'imdb_id':0}],f)
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

startYear:   0%|          | 0/2 [00:00<?, ?it/s]

## **Start Inner Loop**

In [17]:
#Get index and movie id from list
# INNER Loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                                desc=f'Movies from {YEAR}',
                                position=1,
                                leave=True):
    try:
        # Retrieve then data for the movie id
        temp = get_movie_with_rating(movie_id)  
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
            
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2001: 0it [00:00, ?it/s]

### **Itterate through the list of movie IDs and make the calls**

In [18]:
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip",
index=False)

## **After the Loop**

In [19]:
print(f"- Total errors: {len(errors)}")

- Total errors: 0
